In [1]:
!pip install qiskit
!pip install qiskit-aer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 101.1 MB/s eta 0:00:00


In [2]:
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_bloch_multivector
import matplotlib.pyplot as plt


In [3]:
def deutsch_oracle(qc, function_type):
    """
    Implements oracle Uf for Deutsch's Algorithm.
    function_type: 'constant_0', 'constant_1', 'balanced_0', or 'balanced_1'
    """
    if function_type == 'constant_0':
        # f(x) = 0 → Do nothing
        pass
    elif function_type == 'constant_1':
        # f(x) = 1 → Apply X on output qubit
        qc.x(1)
    elif function_type == 'balanced_0':
        # f(x) = x → Apply CNOT
        qc.cx(0, 1)
    elif function_type == 'balanced_1':
        # f(x) = NOT(x) → Apply X, then CNOT, then X
        qc.x(0)
        qc.cx(0, 1)
        qc.x(0)


In [4]:
# Choose function type
function_type = 'balanced_1'  # Try: constant_0, constant_1, balanced_0, balanced_1

# Create circuit with 2 qubits, 1 classical bit
qc = QuantumCircuit(2, 1)

# Initialize |0>|1>
qc.x(1)
qc.barrier()

# Apply Hadamard gates
qc.h([0, 1])
qc.barrier()

# Apply the oracle
deutsch_oracle(qc, function_type)
qc.barrier()

# Apply Hadamard to input qubit
qc.h(0)
qc.barrier()

# Measure input qubit
qc.measure(0, 0)

# Show the circuit
print(qc.draw(output='text'))


           ░ ┌───┐ ░ ┌───┐     ┌───┐ ░ ┌───┐ ░ ┌─┐
q_0: ──────░─┤ H ├─░─┤ X ├──■──┤ X ├─░─┤ H ├─░─┤M├
     ┌───┐ ░ ├───┤ ░ └───┘┌─┴─┐└───┘ ░ └───┘ ░ └╥┘
q_1: ┤ X ├─░─┤ H ├─░──────┤ X ├──────░───────░──╫─
     └───┘ ░ └───┘ ░      └───┘      ░       ░  ║ 
c: 1/═══════════════════════════════════════════╩═
                                                0 


In [5]:
sim = AerSimulator()
compiled_circuit = transpile(qc, sim)
result = sim.run(compiled_circuit).result()
counts = result.get_counts()

print("\nMeasurement Results:", counts)

if list(counts.keys())[0] == '0':
    print("→ Function is CONSTANT.")
else:
    print("→ Function is BALANCED.")



Measurement Results: {'1': 1024}
→ Function is BALANCED.


In [6]:
# Visualize the quantum state after each key stage
qc_state = QuantumCircuit(2)
qc_state.x(1)
qc_state.h([0, 1])

# Bloch sphere after initial Hadamards
state_before_oracle = Statevector(qc_state)
plot_bloch_multivector(state_before_oracle)
plt.show()

# Apply oracle
deutsch_oracle(qc_state, function_type)

# Bloch sphere after oracle
state_after_oracle = Statevector(qc_state)
plot_bloch_multivector(state_after_oracle)
plt.show()


In [7]:
def custom_oracle(qc):
    """
    f(x) = x XOR 1
    Equivalent to flipping the output qubit and applying CNOT
    """
    qc.x(1)
    qc.cx(0, 1)

# Build custom circuit
qc_custom = QuantumCircuit(2, 1)
qc_custom.x(1)
qc_custom.h([0, 1])
custom_oracle(qc_custom)
qc_custom.h(0)
qc_custom.measure(0, 0)

# Simulate
sim = AerSimulator()
compiled_custom = transpile(qc_custom, sim)
result_custom = sim.run(compiled_custom).result()
counts_custom = result_custom.get_counts()

print("\nCustom Oracle Measurement:", counts_custom)
if list(counts_custom.keys())[0] == '0':
    print("→ Function is CONSTANT.")
else:
    print("→ Function is BALANCED.")



Custom Oracle Measurement: {'1': 1024}
→ Function is BALANCED.


In [8]:
# Deutsch–Jozsa for n=2 input qubits

n = 2
qc_dj = QuantumCircuit(n + 1, n)

# Initialize output qubit to |1>
qc_dj.x(n)
qc_dj.h(range(n + 1))

# Example balanced oracle: f(x) = x1 XOR x2
qc_dj.cx(0, n)
qc_dj.cx(1, n)

qc_dj.h(range(n))
qc_dj.measure(range(n), range(n))

print(qc_dj.draw(output='text'))

# Simulate
sim = AerSimulator()
result_dj = sim.run(transpile(qc_dj, sim)).result()
print("Deutsch–Jozsa Results:", result_dj.get_counts())


     ┌───┐          ┌───┐     ┌─┐   
q_0: ┤ H ├───────■──┤ H ├─────┤M├───
     ├───┤       │  └───┘┌───┐└╥┘┌─┐
q_1: ┤ H ├───────┼────■──┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐┌─┴─┐└───┘ ║ └╥┘
q_2: ┤ X ├┤ H ├┤ X ├┤ X ├──────╫──╫─
     └───┘└───┘└───┘└───┘      ║  ║ 
c: 2/══════════════════════════╩══╩═
                               0  1 
Deutsch–Jozsa Results: {'11': 1024}


In [9]:
def classical_check(f):
    """Classical method: must test both inputs."""
    return "BALANCED" if f(0) != f(1) else "CONSTANT"

# Example classical test functions
f1 = lambda x: 0       # constant_0
f2 = lambda x: 1       # constant_1
f3 = lambda x: x       # balanced_0
f4 = lambda x: x ^ 1   # balanced_1

for f in [f1, f2, f3, f4]:
    print(f"Classical check: {classical_check(f)} (needs 2 queries)")
print("Quantum algorithm: only 1 query needed!")


Classical check: CONSTANT (needs 2 queries)
Classical check: CONSTANT (needs 2 queries)
Classical check: BALANCED (needs 2 queries)
Classical check: BALANCED (needs 2 queries)
Quantum algorithm: only 1 query needed!


In [10]:
from qiskit_aer.noise import NoiseModel, depolarizing_error

# Simple depolarizing noise model
noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(depolarizing_error(0.05, 1), ['h', 'x'])
noise_model.add_all_qubit_quantum_error(depolarizing_error(0.1, 2), ['cx'])

# Run noisy simulation
noisy_sim = AerSimulator(noise_model=noise_model)
result_noisy = noisy_sim.run(transpile(qc, noisy_sim)).result()
counts_noisy = result_noisy.get_counts()

print("Noisy Simulation Results:", counts_noisy)


Noisy Simulation Results: {'0': 181, '1': 843}
